# Analyse exploratoire COVID-19

Ce notebook contient les premières analyses et visualisations des données COVID-19.

## Utilisation

1. Assurez-vous d'avoir les données dans `data/raw/covid_data.csv`
2. Exécutez les cellules pour explorer les données
3. Modifiez et expérimentez selon vos besoins



## 1. Configuration et Imports


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Bibliothèques importées avec succès")


## 2. Chargement des Données


In [ ]:
# Chargement du fichier CSV
df = pd.read_csv('../data/raw/covid_data.csv')

print(f"📊 Données chargées : {len(df)} lignes, {len(df.columns)} colonnes")
print(f"\n📅 Colonnes disponibles :")
print(df.columns.tolist())


## 3. Exploration Initiale


In [ ]:
# Aperçu des données
print("### Premières lignes du dataset :")
display(df.head(10))

print("\n### Informations du dataset :")
df.info()

print("\n### Statistiques descriptives :")
display(df.describe())


## 4. Analyse des Valeurs Manquantes


In [ ]:
# Calcul des valeurs manquantes
missing = pd.DataFrame({
    'Colonne': df.columns,
    'Nombre_manquant': df.isnull().sum(),
    'Pourcentage': (df.isnull().sum() / len(df)) * 100
})
missing = missing[missing['Nombre_manquant'] > 0].sort_values('Pourcentage', ascending=False)

print("### Valeurs manquantes par colonne :")
display(missing)

# Visualisation
if len(missing) > 0:
    plt.figure(figsize=(12, 6))
    plt.bar(missing['Colonne'], missing['Pourcentage'], color='coral')
    plt.xticks(rotation=45, ha='right')
    plt.xlabel('Colonnes')
    plt.ylabel('Pourcentage de valeurs manquantes (%)')
    plt.title('Distribution des Valeurs Manquantes')
    plt.tight_layout()
    plt.show()
else:
    print("✅ Aucune valeur manquante détectée")


## 5. Analyse Temporelle


In [ ]:
# Conversion de la colonne date
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    
    print(f"📅 Période couverte : {df['date'].min()} à {df['date'].max()}")
    print(f"   Durée : {(df['date'].max() - df['date'].min()).days} jours")
    
    # Évolution des cas dans le temps
    if 'total_cases' in df.columns:
        plt.figure(figsize=(14, 6))
        
        # Grouper par date
        daily_cases = df.groupby('date')['total_cases'].sum()
        
        plt.plot(daily_cases.index, daily_cases.values, linewidth=2, color='steelblue')
        plt.fill_between(daily_cases.index, daily_cases.values, alpha=0.3)
        plt.xlabel('Date', fontsize=12)
        plt.ylabel('Nombre total de cas', fontsize=12)
        plt.title('Évolution Temporelle des Cas COVID-19', fontsize=14, fontweight='bold')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()


## 6. Analyse Géographique


In [ ]:
if 'location' in df.columns:
    # Top 10 pays par cas
    if 'total_cases' in df.columns:
        top_countries = df.groupby('location')['total_cases'].max().nlargest(10).sort_values(ascending=True)
        
        plt.figure(figsize=(12, 6))
        plt.barh(top_countries.index, top_countries.values, color='crimson')
        plt.xlabel('Nombre total de cas', fontsize=12)
        plt.ylabel('Pays', fontsize=12)
        plt.title('Top 10 Pays - Cas Totaux COVID-19', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    # Statistiques par pays
    print("\n### Statistiques par pays :")
    country_stats = df.groupby('location').agg({
        'total_cases': 'max',
        'total_deaths': 'max' if 'total_deaths' in df.columns else 'count'
    }).sort_values('total_cases', ascending=False).head(10)
    display(country_stats)


## 7. Analyse des Distributions


In [ ]:
# Sélection des colonnes numériques
numeric_cols = df.select_dtypes(include=[np.number]).columns

# Création de distributions pour les principales colonnes
if 'new_cases' in df.columns:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Histogramme
    axes[0, 0].hist(df['new_cases'].dropna(), bins=50, color='skyblue', edgecolor='black')
    axes[0, 0].set_title('Distribution des Nouveaux Cas')
    axes[0, 0].set_xlabel('Nouveaux cas')
    axes[0, 0].set_ylabel('Fréquence')
    
    # Boxplot
    axes[0, 1].boxplot(df['new_cases'].dropna())
    axes[0, 1].set_title('Boxplot des Nouveaux Cas')
    axes[0, 1].set_ylabel('Nouveaux cas')
    
    # Violin plot
    if 'location' in df.columns:
        top_5_countries = df.groupby('location')['new_cases'].max().nlargest(5).index
        df_top5 = df[df['location'].isin(top_5_countries)]
        
        axes[1, 0].violinplot([df_top5[df_top5['location'] == c]['new_cases'].dropna() 
                               for c in top_5_countries], showmeans=True)
        axes[1, 0].set_title('Violin Plot - Top 5 Pays')
        axes[1, 0].set_xticks(range(1, 6))
        axes[1, 0].set_xticklabels(top_5_countries, rotation=45, ha='right')
    
    # QQ plot (approximation)
    try:
        from scipy import stats
        stats.probplot(df['new_cases'].dropna(), dist="norm", plot=axes[1, 1])
        axes[1, 1].set_title('QQ Plot')
    except:
        axes[1, 1].text(0.5, 0.5, 'Scipy non disponible', ha='center', va='center')
        axes[1, 1].set_title('QQ Plot (non disponible)')
    
    plt.tight_layout()
    plt.show()


## 8. Analyse des Corrélations


In [ ]:
# Matrice de corrélation
numeric_df = df.select_dtypes(include=[np.number])

if len(numeric_df.columns) > 1:
    plt.figure(figsize=(12, 10))
    
    corr_matrix = numeric_df.corr()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    
    sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
                cmap='coolwarm', center=0, square=True, 
                linewidths=1, cbar_kws={"shrink": 0.8})
    
    plt.title('Matrice de Corrélation - Variables COVID-19', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Top corrélations
    print("\n### Top 10 Corrélations Positives :")
    corr_pairs = corr_matrix.unstack()
    corr_pairs = corr_pairs[corr_pairs < 1]  # Exclure auto-corrélation
    print(corr_pairs.sort_values(ascending=False).head(10))


## 9. Insights et Observations Clés


In [ ]:
print("### 📊 Résumé de l'Analyse Exploratoire")
print("="*60)

# Statistiques générales
print(f"\n1. DONNÉES GÉNÉRALES :")
if 'date' in df.columns:
    print(f"   • Période analysée : {df['date'].min()} - {df['date'].max()}")
print(f"   • Nombre total d'enregistrements : {len(df):,}")
if 'location' in df.columns:
    print(f"   • Nombre de pays/régions : {df['location'].nunique()}")

# Statistiques COVID
if 'total_cases' in df.columns:
    print(f"\n2. STATISTIQUES COVID :")
    print(f"   • Cas totaux (max) : {df['total_cases'].max():,.0f}")
    if 'total_deaths' in df.columns:
        print(f"   • Décès totaux (max) : {df['total_deaths'].max():,.0f}")
        total_cases_sum = df['total_cases'].sum()
        total_deaths_sum = df['total_deaths'].sum()
        if total_cases_sum > 0:
            mortality = (total_deaths_sum / total_cases_sum) * 100
            print(f"   • Taux de mortalité estimé : {mortality:.2f}%")

# Qualité des données
print(f"\n3. QUALITÉ DES DONNÉES :")
print(f"   • Colonnes avec valeurs manquantes : {(df.isnull().sum() > 0).sum()}")
total_cells = len(df) * len(df.columns)
missing_cells = df.isnull().sum().sum()
completeness = ((total_cells - missing_cells) / total_cells) * 100 if total_cells > 0 else 0
print(f"   • Taux de complétude moyen : {completeness:.1f}%")

print("\n" + "="*60)
print("✅ Analyse exploratoire terminée !")


## 10. Export et Sauvegarde


In [ ]:
# Sauvegarde des données nettoyées
if all(c in df.columns for c in ['date', 'location']):
    df_clean = df.dropna(subset=['date', 'location'])
else:
    df_clean = df.dropna()

df_clean.to_csv('../data/processed/covid_data_clean.csv', index=False)

print(f"✅ Données nettoyées sauvegardées : {len(df_clean)} lignes")
print("📂 Fichier : ../data/processed/covid_data_clean.csv")

print("\n" + "="*60)
print("📊 ANALYSE EXPLORATOIRE COMPLÈTE")
print("="*60)
print("\nProchaines étapes :")
print("1. Utiliser les scripts Python pour le pipeline complet")
print("2. Générer les visualisations avancées")
print("3. Créer le rapport PDF automatisé")
print("\nCommande : python main.py")
